#### Load imports

In [ ]:
import os
import datetime
import time
import math
import numpy as np
import pandas as pd
import geopandas as gpd
from types import SimpleNamespace
import re
import matlab
import altair as alt

from enrolment_geography import *

#### Define directories

In [ ]:
data_dir = os.path.join(os.getcwd(), 'data')
data_dir

In [ ]:
output_dir = os.path.join(os.getcwd(), 'output')
output_dir = create_output_directory(output_dir)
output_dir

In [ ]:
engine = matlab.engine.start_matlab('-desktop')

In [ ]:
postcodes = load_postcodes(os.path.join(data_dir, "postcodes_multi_csv"))

#### Read Enrolments Data:
Postcode, Study Code, Study Name

In [ ]:
enrolments = pd.read_excel(os.path.join(data_dir, "enrolments.xlsx"))

enrolment_stats = SimpleNamespace()
enrolment_stats.n_input = enrolments.shape[0]

enrolment_locations, enrolment_loc_stats = postcodes.lookup_postcodes(
    enrolments['Postcode'], 
    lsoa_field='Boundary Code',
    wgs84_fields=None)

enrolment_stats.n_with_postcodes = \
    enrolment_stats.n_input \
    - enrolment_loc_stats.n_missing_postcodes \
    - enrolment_loc_stats.n_non_geographic_postcodes \
    - enrolment_loc_stats.n_unknown_postcodes

enrolments = pd.concat((enrolments.drop(columns=['Postcode']), 
                        enrolment_locations),
                        axis=1)

print("\n".join(enrolments.columns))
print("Missing study names: {:d}".format((enrolments['Study Name'].isnull() | (enrolments['Study Name'] == '')).sum()))
enrolments.shape

#### Map enrolment specialities

In [ ]:
enrolment_specialities = pd.read_excel(os.path.join(data_dir, "specialities.xlsx"))
enrolment_specialities = enrolment_specialities.drop(columns=['URL'])
enrolment_specialities['Speciality'].fillna('???', inplace=True)
unique_specialities = enrolment_specialities.loc[:, 'Speciality'].unique()
print("\n".join(unique_specialities))

In [ ]:
enrolments = enrolments.merge(enrolment_specialities, how='left', on='Study Name')
enrolments = enrolments.reindex(columns=[col for col in enrolments.columns if col != "Boundary Code"] + ["Boundary Code"])
print("\n".join(enrolments.columns))
enrolments.shape

In [ ]:
enrolments['Study Name'].unique().shape

In [ ]:
enrolments_by_speciality = enrolments.groupby('Speciality')
enrolment_stats.n_with_speciality = enrolments_by_speciality.size().sum()

In [ ]:
enrolments_by_speciality.size().sum()

In [ ]:
assert enrolment_stats.n_input == enrolment_stats.n_with_speciality, "Enrolments with missing speciality"

#### Load Census Geometry

In [ ]:
lsoa_polygons = gpd.read_file(os.path.join(data_dir, "lsoa_boundaries/Lower_Layer_Super_Output_Areas_December_2011_Boundaries_EW_BGC.shp"))
lsoa_polygons = lsoa_polygons.to_crs('EPSG:27700')
lsoa_polygons.rename(columns={'LSOA11CD': 'Boundary Code', 'LSOA11NM': 'Boundary Name', 'geometry': "Boundary Polygon"}, inplace=True)
lsoa_polygons = lsoa_polygons.drop(columns=['Age_Indica', 'Shape__Are', 'Shape__Len', 'LSOA11NMW', 'FID'])
lsoa_polygons.insert(len(lsoa_polygons.columns), 'Boundary Centroid', lsoa_polygons['Boundary Polygon'].centroid)
print("\n".join(lsoa_polygons.columns))
lsoa_polygons.shape

In [ ]:
assert lsoa_polygons['Boundary Code'].nunique() == lsoa_polygons.shape[0], "Polygon data does not contain unique boundary codes"

In [ ]:
lsoa_m25_polygons = gpd.read_file(os.path.join(data_dir, "m25_lsoa11.gpkg"))
lsoa_m25_polygons = lsoa_m25_polygons.to_crs('EPSG:27700')
lsoa_m25_polygons.rename(columns={'LSOA11CD': 'Boundary Code', 'LSOA11NM': 'Boundary Name'}, inplace=True)
lsoa_m25_polygons = lsoa_m25_polygons.drop(columns=['Age_Indica', 'Shape__Are', 'Shape__Len', 'LSOA11NMW', 'geometry'])
print("\n".join(lsoa_m25_polygons.columns))
lsoa_m25_polygons.shape

#### Load Deprivation data for Census Geometry

In [ ]:
lsoa_deprivation = pd.read_csv(os.path.join(data_dir, "imd2019lsoa_scores.csv"))
lsoa_deprivation = lsoa_deprivation.drop(columns=[col for col in lsoa_deprivation.columns if len(lsoa_deprivation[col].unique()) <= 1])
lsoa_deprivation.rename(columns={'LSOA11': 'Boundary Code', 'Reference area': 'Boundary Name'}, inplace=True)

deprivation_indices = lsoa_deprivation.columns[2:]
print("\n".join(lsoa_deprivation.columns))
lsoa_deprivation.shape

#### Load population densities

In [ ]:
lsoa_densities = pd.read_excel(os.path.join(data_dir, "SAPE22DT11-mid-2019-lsoa-population-density.xlsx"), header=4, sheet_name=3)
lsoa_densities = lsoa_densities.drop(columns=[col for col in lsoa_densities.columns if len(lsoa_densities[col].unique()) <= 1])
lsoa_densities.rename(columns={'LSOA Code': 'Boundary Code', 'LSOA Name': 'Boundary Name', 'Mid-2019 population': 'N Population', 'Area Sq Km': 'Area km^2', 'People per Sq Km': 'Population per km^2'}, inplace=True)
print("\n".join(lsoa_densities.columns))
lsoa_densities.shape

In [ ]:
def compute_min_distance_from(location, gp_prev):

    min_distance = None
    min_distance_index = None

    for index in range(gp_prev.shape[0]):

        gp_centroid = gp_prev["Boundary Centroid"][index]

        distance = location.distance(gp_centroid)

        if min_distance is None or min_distance > distance:
            min_distance = distance
            min_distance_index = index
    
    return min_distance, min_distance_index
    
def impute_missing_lsoas(gp_prev, boundaries):
    
    gp_prev = gp_prev.sample(frac=1, random_state=0xFBEB6693).reset_index(drop=True)
    gp_prev['Boundary Centroid'] = gpd.GeoSeries.from_wkt(gp_prev['Boundary Centroid'])
    gp_prev = gpd.GeoDataFrame(gp_prev, geometry='Boundary Centroid', crs='EPSG:27700')
    
    imputed_gp_prev = []
    gp_prev_lsoas = gp_prev.groupby('Boundary Code')
    
    match_count = 0
    
    for index, code in enumerate(boundaries['Boundary Code']):
        
        # print(f"[{index:d}]: {code}")
        
        indices = gp_prev_lsoas.indices.get(code, None)
        
        if indices is None:
            indices = list()
        else:
            indices = indices.tolist()
        
        assert len(indices) < 2, "Expecting at most one row per boundary code in gp_prev."
        
        # For some reason if indices: evaluates to False for exactly one list instance with length 1?!?
        
        if len(indices) != 0:
            match_count += 1
            
            gp_prev_row = gp_prev.iloc[indices[0], :]
            gp_code = gp_prev_row['Boundary Code']
            assert gp_code == code, f"Boundary code does not match its index: {gp_code} != {code}"
            
            continue
        
        centroid = boundaries.at[index, 'Boundary Centroid']
        
        _, min_distance_index = compute_min_distance_from(centroid, gp_prev)
        
        matched_row = gp_prev.iloc[min_distance_index, :].to_dict()
        matched_row['Boundary Code'] = code
        matched_row['Boundary Centroid'] = centroid
        
        imputed_gp_prev.append(matched_row)
    
    print(f"Match count = {match_count:d}")
    
    imputed_gp_prev = pd.DataFrame(imputed_gp_prev)
    
    result = pd.concat((gp_prev, imputed_gp_prev), ignore_index=True)
    
    assert len(result.groupby('Boundary Code')) == result.shape[0], "Result gp_prev contains duplicate boundary codes."

    return result


#### Map Quality Outcomes Framework Data

In [ ]:
use_cached_gp_prev = False # Enable caching of data if it already has been pre-computed

In [ ]:
if not use_cached_gp_prev:
    gp_prev = pd.read_csv(os.path.join(data_dir, "qof_prevalence_2020.csv"), index_col=0)
    assert gp_prev['Boundary Code'].nunique() == gp_prev.shape[0], "QOF data does not contain unique boundary codes"
    print(gp_prev.shape)
    gp_prev.drop(columns=[col for col in gp_prev.columns if col.startswith("List Size") or col.startswith("Register") or col == "Boundary Name"], inplace=True)
    gp_prev = impute_missing_lsoas(gp_prev, lsoa_polygons)
    gp_prev.drop(columns=["OSGB36 Location", "Boundary Polygon", "Boundary Centroid", "Postcode", "Min Distance"], inplace=True)
    print("\n".join(gp_prev.columns))
    print(gp_prev.shape)
    assert gp_prev['Boundary Code'].nunique() == gp_prev.shape[0], "QOF data does not contain unique boundary codes"
    gp_prev.to_csv(os.path.join(output_dir, "gp_prev_interp.csv"), index=True)

In [ ]:
if use_cached_gp_prev:
    gp_prev = pd.read_csv(os.path.join(data_dir, "gp_prev_interp.csv"), index_col=0)

In [ ]:
gp_prev.shape

In [ ]:
lsoa = lsoa_densities.merge(lsoa_deprivation.drop(columns=['Boundary Name']), how='inner', on='Boundary Code')
lsoa = lsoa.merge(gp_prev, how='left', on='Boundary Code')
# lsoa['Average Prevalence'] = 1.0
lsoa_stats = SimpleNamespace()
lsoa_stats.n_initial = lsoa.shape[0]

In [ ]:
# assert (lsoa.shape[0] - lsoa['Average Prevalence'].isnull().sum()) == gp_prev.shape[0], "Not all QOF data matched by LOSAs."

In [ ]:
# Expressses the prevalences relative to the LSOA population
def adjust_prevalences(data, field='N Population', factor=10000.0):
    
    for column in list(data.columns):

        if not column.endswith('Prevalence'):
            continue
        
        data[column + " per 10k"] = np.floor(data[column] * data[field] * factor)
    

In [ ]:
adjust_prevalences(lsoa)

print("\n".join(lsoa.columns))
lsoa.shape

#### Compute Enrolment Stats

In [ ]:
lsoa_enrolments = enrolments.merge(lsoa, how='inner', on='Boundary Code')

In [ ]:
enrolment_stats.n_with_lsoas = lsoa_enrolments.shape[0]

In [ ]:
if enrolment_stats.n_with_postcodes != enrolment_stats.n_with_lsoas:
    enrolment_codes = set(enrolments['Boundary Code'])
    lsoa_codes = set(lsoa['Boundary Code'])
    missing_enrolment_codes = enrolment_codes.difference(lsoa_codes)
    print(f"{(lsoa_polygons['Boundary Code'].isin(missing_enrolment_codes)).sum():d}")
    print(f"LSOA enrolments now contain {lsoa_enrolments.shape[0]:d} rows.")
    print(f"Unresolved LSOAs in enrolments [{len(missing_enrolment_codes)}]:")
    print(missing_enrolment_codes)

In [ ]:
lsoa_enrolments_by_speciality = lsoa_enrolments[~(lsoa_enrolments['Boundary Code'].isnull() | lsoa_enrolments['Speciality'].isnull())].groupby('Speciality')
lsoa_enrolments_by_speciality.size()

In [ ]:
lsoa_enrolment_study_counts = count_group_values(lsoa_enrolments, 
                                           'Study Name', 'N Participants')
lsoa_enrolment_study_counts

In [ ]:
lsoa_enrolment_study_counts = lsoa_enrolment_study_counts.merge(enrolment_specialities, how='left', on='Study Name')
lsoa_enrolment_study_counts

In [ ]:
lsoa_enrolment_studies = count_group_values(lsoa_enrolment_study_counts, 
                                           'Speciality', 'N Studies',
                                            None,
                                            [('Study Name', 'Study Name'), ('N Participants', 'N Participants')])

In [ ]:
lsoa_enrolment_studies['N Participants'] = lsoa_enrolment_studies.apply(
    lambda row: np.array(row['N Participants']).sum(), 
    axis=1,
    result_type="reduce")
lsoa_enrolment_studies

In [ ]:
unique_studies = set(enrolments['Study Name'].unique())
unique_studies_lsoa = set(lsoa_enrolments['Study Name'].unique())
missing_studies = list(unique_studies.difference(unique_studies_lsoa))
missing_studies

In [ ]:
enrolments.loc[enrolments['Study Name'] == missing_studies[1], 'Boundary Code']

#### Compute participant and study counts per census area

In [ ]:
lsoa_enrolment_counts = count_group_values(lsoa_enrolments, 
                                           'Boundary Code', 'N Participants',
                                           [('Study Name', 'N Studies')],
                                           [('Speciality', 'Speciality')])
lsoa_enrolment_counts

In [ ]:
assert lsoa_enrolment_counts['N Participants'].sum() == enrolment_stats.n_with_lsoas, "N Participants sum does not match number of enrolments with LSOAs."

In [ ]:
enrolment_stats

#### Compute speciality indicators

In [ ]:
speciality_indicators = lsoa_enrolment_counts.apply(
    lambda row: expand_indicators(row, "Speciality", unique_specialities, "Speciality"), 
    axis=1,
    result_type="expand")

In [ ]:
lsoa_enrolment_counts = pd.concat([lsoa_enrolment_counts.drop(columns=['Speciality']), speciality_indicators], axis=1)
print("\n".join(lsoa_enrolment_counts.columns))
lsoa_enrolment_counts.shape

#### Add count and speciality information to enrolment areas

In [ ]:
enrolment_lsoas = lsoa.merge(lsoa_enrolment_counts, how='left', on='Boundary Code')
enrolment_lsoas['N Participants'] = enrolment_lsoas['N Participants'].fillna(0)
enrolment_lsoas['N Studies'] = enrolment_lsoas['N Studies'].fillna(0)
enrolment_lsoas['N per 10k'] = np.floor((enrolment_lsoas['N Participants'] / enrolment_lsoas['N Population'] * 10000.0))
enrolment_lsoas['N per 10k Need-Adjusted'] = np.floor((enrolment_lsoas['N Participants'] / (enrolment_lsoas['Average Prevalence'] * enrolment_lsoas['N Population']) * 10000.0))

for column in enrolment_lsoas.columns:
    if column.endswith('[Speciality]'):
        enrolment_lsoas[column] = enrolment_lsoas[column].fillna(0)

print("\n".join(enrolment_lsoas.columns))
enrolment_lsoas.shape

In [ ]:
# enrolment_lsoas.plot.line(y=['N per 10k Need-Adjusted', 'N per 10k Need-Adjusted Actual'])

In [ ]:
columns = set(enrolment_lsoas.columns)

#### Add region indicator to enrolment areas

In [ ]:
enrolment_lsoas.insert(enrolment_lsoas.shape[1], 'Region', '')
enrolment_lsoas.loc[enrolment_lsoas['Boundary Code'].isin(lsoa_m25_polygons['Boundary Code'].unique()), 'Region'] = 'M25'

In [ ]:
enrolment_lsoas = gpd.GeoDataFrame(
    enrolment_lsoas.merge(lsoa_polygons.drop(columns=["Boundary Name"]), how='inner', on='Boundary Code'),
    crs='EPSG:27700',
    geometry="Boundary Polygon")

In [ ]:
enrolment_lsoas.dtypes

In [ ]:
save_enrolment_polygons = False

In [ ]:
if save_enrolment_polygons:
    enrolment_lsoa_polygons = enrolment_lsoas.drop(columns=["Boundary Centroid"])
    enrolment_lsoa_polygons.to_file("enrolment_lsoa_polygons.gpkg", driver='GPKG', layer='enrolment_lsoa_polygons')  

#### Randomly reshuffle enrolment areas

In [ ]:
enrolment_lsoas = enrolment_lsoas.sample(frac=1, random_state=0x80728334).reset_index(drop=True)

#### Define data subsets and collect stats: enrolment vs no enrolment and without and with QOF data

In [ ]:
enrolment_lsoas_eq_0 = enrolment_lsoas[enrolment_lsoas['N Participants'] == 0]
enrolment_lsoas_ge_1 = enrolment_lsoas[enrolment_lsoas['N Participants'] >= 1]

In [ ]:
lsoa_stats.n_without_participants = enrolment_lsoas_eq_0.shape[0]
lsoa_stats.n_with_participants = enrolment_lsoas_ge_1.shape[0]
lsoa_stats.n_with_participants_sum = int(enrolment_lsoas_ge_1['N Participants'].sum())

assert lsoa_stats.n_initial == (lsoa_stats.n_with_participants + lsoa_stats.n_without_participants), "Inconsistency bewteen LSOAs with participants and without."

In [ ]:
lsoa_stats.n_greater_london_initial = enrolment_lsoas[enrolment_lsoas['Region'] == "M25"].shape[0]
lsoa_stats.n_greater_london_without_participants = enrolment_lsoas[(enrolment_lsoas['Region'] == "M25") & (enrolment_lsoas['N Participants'] == 0)].shape[0]
lsoa_stats.n_greater_london_with_participants = enrolment_lsoas[(enrolment_lsoas['Region'] == "M25") & (enrolment_lsoas['N Participants'] > 0)].shape[0]
lsoa_stats.n_greater_london_with_participants_sum = int(enrolment_lsoas[(enrolment_lsoas['Region'] == "M25") & (enrolment_lsoas['N Participants'] > 0)]['N Participants'].sum())

In [ ]:
enrolment_lsoas_gp = enrolment_lsoas.dropna(subset=['Average Prevalence'])

In [ ]:
enrolment_lsoas_gp_eq_0 = enrolment_lsoas_gp[enrolment_lsoas_gp['N Participants'] == 0]
enrolment_lsoas_gp_ge_1 = enrolment_lsoas_gp[enrolment_lsoas_gp['N Participants'] >= 1]

In [ ]:
lsoa_stats.n_gp_initial = enrolment_lsoas_gp.shape[0]
lsoa_stats.n_gp_without_participants = enrolment_lsoas_gp_eq_0.shape[0]
lsoa_stats.n_gp_with_participants = enrolment_lsoas_gp_ge_1.shape[0]
lsoa_stats.n_gp_with_participants_sum = int(enrolment_lsoas_gp_ge_1['N Participants'].sum())

In [ ]:
lsoa_stats.n_gp_greater_london_initial = enrolment_lsoas_gp[enrolment_lsoas_gp['Region'] == "M25"].shape[0]
lsoa_stats.n_gp_greater_london_without_participants = enrolment_lsoas_gp[(enrolment_lsoas_gp['Region'] == "M25") & (enrolment_lsoas_gp['N Participants'] == 0)].shape[0]
lsoa_stats.n_gp_greater_london_with_participants = enrolment_lsoas_gp[(enrolment_lsoas_gp['Region'] == "M25") & (enrolment_lsoas_gp['N Participants'] > 0)].shape[0]
lsoa_stats.n_gp_greater_london_with_participants_sum = int(enrolment_lsoas_gp[(enrolment_lsoas_gp['Region'] == "M25") & (enrolment_lsoas_gp['N Participants'] > 0)]['N Participants'].sum())

In [ ]:
def save_namespace(ns, name, file_path):
    ns_dict = vars(ns)
    
    index = list(ns_dict.keys())
    values = []
    
    for key in index:
        values.append(int(ns_dict[key]))
    
    ns_df = pd.DataFrame(values, index=index, columns=[name])
    ns_df.to_csv(file_path + ".csv")
    

In [ ]:
stats_dir = output_dir

In [ ]:
save_namespace(lsoa_stats, "lsoas", os.path.join(stats_dir, "lsoa_stats"))

In [ ]:
save_namespace(enrolment_stats, "enrolments", os.path.join(stats_dir, "enrolment_stats"))

#### Prepare regression models

In [ ]:
identifier_map = {
    "Index of Multiple Deprivation (IMD)": "imd",
    "Health Deprivation and Disability Domain": "health",
    "Living Environment Deprivation Domain": "environment",
    "Education, Skills and Training Domain": "education",
    "Income Deprivation Domain": "income",
    "Employment Deprivation Domain": "employment",
    "Crime Domain": "crime",
    "Barriers to Housing and Services Domain": "housing_barriers",
    "Income Deprivation Affecting Children Index (IDACI)": "income_affecting_children",
    "Income Deprivation Affecting Older People Index (IDAOPI)": "income_affecting_older_people"
}

In [ ]:
def standardise_transform(response, variable_name):
    transform = TransformFunction()
    transform.identifier_map.update(identifier_map)
    transform.select = [response, variable_name]
    transform.apply_to.add(variable_name)
    transform.function = lambda x: (x - np.mean(x)) / np.std(x)
    return transform

In [ ]:
def select_transform(*variable_names):
    transform = TransformFunction()
    transform.identifier_map.update(identifier_map)
    transform.select.extend(variable_names)
    return transform

In [ ]:
for variable_name in list(deprivation_indices):
    print("{}: [{:.2f}, {:.2f}]".format(variable_name, enrolment_lsoas[variable_name].min(), enrolment_lsoas[variable_name].max()))

In [ ]:
# Define suitable x ranges for the deprivation data

deprivation_params = {
"N per 10k":
{},
    
"Index of Multiple Deprivation (IMD)":
{ "x_scale": alt.Scale(domain=[0, 100]), "x_step": 1.0},
    
"Income Deprivation Domain":
{ "x_scale": alt.Scale(domain=[0, 1]), "x_step": 0.01},
    
"Employment Deprivation Domain":
{ "x_scale": alt.Scale(domain=[0, 1]), "x_step": 0.01},
    
"Education, Skills and Training Domain":
{ "x_scale": alt.Scale(domain=[0, 100]), "x_step": 1.0},
    
"Health Deprivation and Disability Domain":
{ "x_scale": alt.Scale(domain=[-4, 4]), "x_step": 0.08},
    
"Crime Domain":
{ "x_scale": alt.Scale(domain=[-4, 4]), "x_step": 0.08},
    
"Barriers to Housing and Services Domain":
{ "x_scale": alt.Scale(domain=[0, 100]), "x_step": 1.0},
    
"Living Environment Deprivation Domain":
{ "x_scale": alt.Scale(domain=[0, 100]), "x_step": 1.0},
    
"Income Deprivation Affecting Children Index (IDACI)":
{ "x_scale": alt.Scale(domain=[0, 1]), "x_step": 0.01},
    
"Income Deprivation Affecting Older People Index (IDAOPI)":
{ "x_scale": alt.Scale(domain=[0, 1]), "x_step": 0.01},
}

In [ ]:
model = 'poisson'
n_samples=3000000
n_burn_in_samples=1000000

regression_tasks = [Regression(None, 
               transforms=[standardise_transform("N per 10k", variable_name)], 
               model=model, 
               n_samples=n_samples, n_burn_in_samples=n_burn_in_samples, 
               prior='ridge',
               display_odds_ratio=True,
               basename=identifier_map[variable_name],
               test_split=None) for variable_name in list(deprivation_indices)]

regression_tasks_need_adjusted = [Regression(None,
               transforms=[standardise_transform("N per 10k Need-Adjusted", variable_name)], 
               model=model, 
               n_samples=n_samples, n_burn_in_samples=n_burn_in_samples, 
               prior='ridge',
               display_odds_ratio=True,
               basename=identifier_map[variable_name] + "_adjusted",
               test_split=None) for variable_name in list(deprivation_indices)]

def akde_arguments(params):
    scale = params["x_scale"].domain
    steps = (scale[1] - scale[0]) / params["x_step"]
    
    result = {
        "estimation_range": scale,
        "estimation_steps": math.ceil(steps * 2.5)
    }
    
    return result

dist_tasks = [DistributionTask(engine, 
                   transforms=[select_transform(variable_name)],
                   basename='_'.join(variable_name.lower().split()) + "_akde",
                   gamma=lambda x: math.ceil(math.pow(x, 1.0 / 2.0)), # lambda x: x, # math.ceil(math.pow(x, 1.0 / 2.0)),
                   estimation_method="akde", # "kde",
                   **akde_arguments(deprivation_params[variable_name]))
    
              for variable_name in list(deprivation_indices)
]

histogram_tasks = [
    RenderHistogram(transforms=[select_transform(variable_name)],
                    basename='_'.join(variable_name.lower().split()) + "_histograms.html",
                    **(deprivation_params[variable_name]))
    
    for variable_name in list(deprivation_indices)
]

#### Lookup UCLH location for distance matching

In [ ]:
uclh_locations, _ = postcodes.lookup_postcodes(
    pd.Series(["NW1 2BU"]), 
    lsoa_field='Boundary Code',
    wgs84_fields=None)
uclh_location = uclh_locations.loc[0, "OSGB36 Easting"], uclh_locations.loc[0, "OSGB36 Northing"]
uclh_location

#### Match distances for areas in England with no enrolment to areas with at least one enrolment

In [ ]:
match_distances = MatchDistances(Domain('England, N >= 1', enrolment_lsoas_ge_1), uclh_location, "Boundary Centroid")

In [ ]:
contexts = run_domain_tasks([Domain('England, N == 0', enrolment_lsoas_eq_0)], [match_distances], output_dir)

In [ ]:
match_path = os.path.join(output_dir, "england,_n_==_0_matched.csv")
enrolment_lsoas_eq_0_matched = pd.read_csv(match_path, index_col=0, dtype={'Region': 'str'})

In [ ]:
enrolment_lsoas_eq_0_matched = gpd.GeoDataFrame(enrolment_lsoas_eq_0_matched)

In [ ]:
enrolment_lsoas_eq_0_matched.dtypes

In [ ]:
enrolment_lsoas_eng = pd.concat([enrolment_lsoas_ge_1, enrolment_lsoas_eq_0_matched], axis=0, ignore_index=True)

In [ ]:
regress_only_ge_1 = False

In [ ]:
domain_eng = Domain('England', enrolment_lsoas_ge_1 if regress_only_ge_1 else enrolment_lsoas_eng)

In [ ]:
domain_lon = Domain('London', enrolment_lsoas_ge_1[enrolment_lsoas_ge_1['Region'] == 'M25'] \
                              if regress_only_ge_1 else \
                              enrolment_lsoas[enrolment_lsoas['Region'] == 'M25'])

In [ ]:
domain_eng.dataset.columns

#### Collect stats about regression models

In [ ]:
regression_distributions = []

for variable_name in ["N per 10k", "N per 10k Need-Adjusted"]:
    
    for domain in [domain_lon, domain_eng]:
        m = np.mean(domain.dataset[variable_name])
        v = np.var(domain.dataset[variable_name])
        
        regression_distributions.append((variable_name, domain.name, m, v))

regression_distributions = pd.DataFrame(regression_distributions, columns=["Response", "Region", "Mean", "Variance"])
regression_distributions

#### Regression: N per 10k vs Deprivation Indices

In [ ]:
contexts = run_domain_tasks([domain_eng, domain_lon], regression_tasks, output_dir)

In [ ]:
match_distances = MatchDistances(Domain('England, N >= 1', enrolment_lsoas_gp_ge_1), uclh_location, "Boundary Centroid", "matched_gp")

In [ ]:
contexts = run_domain_tasks([Domain('England, N == 0', enrolment_lsoas_gp_eq_0)], [match_distances], output_dir)

In [ ]:
match_path = os.path.join(output_dir, "england,_n_==_0_matched_gp.csv")
enrolment_lsoas_gp_eq_0_matched = pd.read_csv(match_path, index_col=0, dtype={'Region': 'str'})

In [ ]:
enrolment_lsoas_gp_eq_0_matched = gpd.GeoDataFrame(enrolment_lsoas_gp_eq_0_matched)

In [ ]:
enrolment_lsoas_eng_gp = pd.concat([enrolment_lsoas_gp_ge_1, enrolment_lsoas_gp_eq_0_matched], axis=0, ignore_index=True)

In [ ]:
domain_eng_gp = Domain('England', enrolment_lsoas_eng_gp if not regress_only_ge_1 else enrolment_lsoas_gp_ge_1)

In [ ]:
domain_lon_gp = Domain('London', enrolment_lsoas_eng_gp[enrolment_lsoas_eng_gp['Region'] == 'M25'] \
                                 if not regress_only_ge_1 else \
                                 enrolment_lsoas_gp_ge_1[enrolment_lsoas_gp_ge_1['Region'] == 'M25'])

In [ ]:
domain_lon_gp.dataset['N Participants'].sum()

#### Regression: N per 10k Need-Adjusted vs Deprivation Indices

In [ ]:
contexts = run_domain_tasks([domain_lon_gp, domain_eng_gp], regression_tasks_need_adjusted, output_dir)

In [ ]:
domain_lon.dataset = domain_lon.dataset.astype({'N per 10k': 'int32'})

In [ ]:
domain_eng.dataset = domain_eng.dataset.astype({'N per 10k': 'int32'})

In [ ]:
domain_eng.dataset.iloc[:, 1].to_numpy().reshape((-1, 1)).shape

#### Compute densities of sub data sets

In [ ]:
# For the Mann Whitney U test the order of the domains matters: The statistic is computed relative to the
# first domain.

domains_eng = [

    Domain('England, N >= 1', enrolment_lsoas_ge_1),
    Domain('England, N == 0', enrolment_lsoas_eq_0_matched),
]

domains_lon = [
    Domain('London, N >= 1', enrolment_lsoas_ge_1[enrolment_lsoas_ge_1['Region'] == 'M25']),
    Domain('London, N == 0', enrolment_lsoas_eq_0[enrolment_lsoas_eq_0['Region'] == 'M25']),
]

In [ ]:
distribution_eng = Domain('England', pd.concat((domains_eng[0].dataset, domains_eng[1].dataset)))
distribution_lon = Domain('London', pd.concat((domains_lon[0].dataset, domains_lon[1].dataset)))

In [ ]:
densities_dir = os.path.join(output_dir, "densities_akde")
os.makedirs(densities_dir, exist_ok=False)

In [ ]:
contexts = run_domain_tasks(domains_lon + domains_eng, dist_tasks, densities_dir)

#### Compute deprivation index ranges

In [ ]:
range_eng = []
range_lon = []
range_percentiles = [0.005, 0.995]

for variable_name in list(deprivation_indices):
    value      = np.nanquantile(distribution_eng.dataset[variable_name], range_percentiles).tolist()
    value_ge_1 = np.nanquantile(domains_eng[0].dataset[variable_name], range_percentiles).tolist()
    value_eq_0 = np.nanquantile(domains_eng[1].dataset[variable_name], range_percentiles).tolist()
    range_eng.append((variable_name, *value, *value_ge_1, *value_eq_0))
    
    value      = np.nanquantile(distribution_lon.dataset[variable_name], range_percentiles).tolist()
    value_ge_1 = np.nanquantile(domains_lon[0].dataset[variable_name], range_percentiles).tolist()
    value_eq_0 = np.nanquantile(domains_lon[1].dataset[variable_name], range_percentiles).tolist()
    range_lon.append((variable_name, *value, *value_ge_1, *value_eq_0))

range_eng = pd.DataFrame(range_eng, columns=['term', 'c005', 'c995', 'c005_ge_1', 'c995_ge_1', 'c005_eq_0', 'c995_eq_0'])
range_lon = pd.DataFrame(range_lon, columns=['term', 'c005', 'c995', 'c005_ge_1', 'c995_ge_1', 'c005_eq_0', 'c995_eq_0'])

In [ ]:
range_eng.to_csv(os.path.join(output_dir, 'range_eng.csv'))
range_eng

In [ ]:
range_lon.to_csv(os.path.join(output_dir, 'range_lon.csv'))
range_lon

#### Compute statistic tests to compare distributional means of sub data sets

In [ ]:
test_transform = TransformFunction()
test_transform.select = list(deprivation_indices)

test_alpha = 0.05
test_task = TestTask(
    basename="tests",
    alpha=test_alpha,
    alpha_corrected=test_alpha / len(deprivation_indices),
    disjoint_test=False,
    test_type="mannwhitneyu", # "ks" "t"
    test_alternative="two-sided",
    transforms=[test_transform]
)

In [ ]:
contexts = run_domain_tasks(domains_lon, [test_task], output_dir)
os.rename(os.path.join(output_dir, 'test_results.csv'), os.path.join(output_dir, 'test_results_london_mwu.csv'))
contexts = run_domain_tasks(domains_eng, [test_task], output_dir)
os.rename(os.path.join(output_dir, 'test_results.csv'), os.path.join(output_dir, 'test_results_england_mwu.csv'))

#### Plot distributions with all information for London and England

In [ ]:
converters = {
    "values": lambda value: eval(value),
    "density": lambda value: eval(value),
    "frequencies": lambda value: eval(value)
}

In [ ]:
def find_files(directory, suffix=None, ext=".csv"):
    
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.endswith(ext)]
    files = sorted(files)
    
    result = []
    
    for file in files:
        path = os.path.join(directory, file)
        file_name, _ = os.path.splitext(file)
        
        if suffix is not None and not file_name.endswith(suffix):
            continue
        
        result.append(path)
    
    return result

In [ ]:
def read_distributions(directory):
    file_paths = find_files(directory, ext=".csv")
    distributions = [pd.read_csv(csv_file, index_col=0, converters=converters) for csv_file in file_paths]
    distributions = pd.concat(distributions, ignore_index=True)
    return distributions

In [ ]:
label_map = {
    "Index of Multiple Deprivation (IMD)": "Index of Multiple\nDeprivation (IMD)",
    "Health Deprivation and Disability": "Health Deprivation\nand Disability",
    "Living Environment Deprivation": "Living Environment\nDeprivation",
    "Education, Skills and Training": "Education, Skills\nand Training",
    "Income Deprivation": "Income Deprivation",
    "Employment Deprivation": "Employment Deprivation",
    "Crime": "Crime",
    "Barriers to Housing and Services": "Barriers to Housing\nand Services",
    "Income Deprivation Affecting Children Index (IDACI)": "Income Deprivation Affecting\nChildren Index (IDACI)",
    "Income Deprivation Affecting Older People Index (IDAOPI)": "Income Deprivation Affecting\nOlder People Index (IDAOPI)"
}

In [ ]:
distributions = read_distributions(densities_dir)
distributions = distributions.replace(to_replace=r'^@"(.+?)( Domain)?"$', value=r'\g<1>', regex=True)
distributions = distributions.replace(to_replace=label_map)
distributions

In [ ]:
distributions_eng = distributions[(distributions['domain'] == "England, N >= 1") | 
                                  (distributions['domain'] == "England, N == 0")]
distributions_lon = distributions[(distributions['domain'] == "London, N >= 1") | 
                                  (distributions['domain'] == "London, N == 0")]

distribution_domains_eng = Domain('Distributions England', distributions_eng)
distribution_domains_lon = Domain('Distributions London', distributions_lon)

In [ ]:
plot_sort_order = [
    "Index of Multiple Deprivation (IMD)",
    "Barriers to Housing and Services",
    "Crime",
    "Education, Skills and Training",
    "Employment Deprivation",
    "Health Deprivation and Disability",
    "Income Deprivation",
    "Income Deprivation Affecting Children Index (IDACI)",
    "Income Deprivation Affecting Older People Index (IDAOPI)",
    "Living Environment Deprivation"
]

plot_sort_order = [label_map[label] for label in plot_sort_order]

In [ ]:
test_results_lon = pd.read_csv(os.path.join(output_dir, "test_results_london_mwu.csv"), index_col=0)
test_results_lon = test_results_lon.replace(to_replace=r'^@"(.+?)( Domain)?"$', value=r'\g<1>', regex=True)
test_results_lon = test_results_lon.replace(to_replace=label_map)

plot_distributions = PlotDistributionsTask(
    "distributions_london_mwu.html",
    use_frequency_scale=True,
    align_scales_across_plots=True,
    align_scales_across_variants=False,
    plot_field="term",
    plot_sort_order=plot_sort_order,
    test_results=None, # test_results_lon,
    grid_columns=2,
    color_mapping={
      "London, N == 0": "#CDC134",  # B2A72D F1F117 E64444 D8C300
      "London, N >= 1": "#36B28D", # 32A683 A968E6
    
      "England, N == 0": "#CDC134",  # B2A72D F1F117 E64444 D8C300
      "England, N >= 1": "#36B28D" # 32A683 A968E6
    },
    interpolate="monotone",
    legend_field="domain",
    legend_view_style="legend-view",
    y_title="Frequency",
    alpha_suffix=", 2s",
    quartile_stroke_dash=[2, 1],
    include_legend=False,
    legend_concat_horizontally=False,
    legend_padding_outer=1,
    include_histograms=True,
    include_rules=True,
    significant_difference_color="#111111",
    insignificant_difference_color="#A62109", #"#DA2B0D", #"#DC462C",
    difference_label_align="left",
    difference_label_baseline="middle",
    difference_label_y=alt.Undefined,
    difference_label_style="result-text",
    variant_order=["London, N == 0", "England, N == 0", "London, N >= 1", "England, N >= 1"],
    vega_config_paths={"light": os.path.join(data_dir, "press-theme-light.json"), 
                       "dark": os.path.join(data_dir, "press-theme-dark.json")})

In [ ]:
contexts = run_domain_tasks([distribution_domains_lon], [plot_distributions], output_dir)

In [ ]:
test_results_eng = pd.read_csv(os.path.join(output_dir, "test_results_england_mwu.csv"), index_col=0)
test_results_eng = test_results_eng.replace(to_replace=r'^@"(.+?)( Domain)?"$', value=r'\g<1>', regex=True)
test_results_eng = test_results_eng.replace(to_replace=label_map)

plot_distributions = PlotDistributionsTask(
    "distributions_england_mwu.html",
    use_frequency_scale=True,
    align_scales_across_plots=True,
    align_scales_across_variants=False,
    plot_field="term",
    plot_sort_order=plot_sort_order,
    test_results=None, # test_results_eng,
    grid_columns=2,
    color_mapping={
      "London, N == 0": "#CDC134",  # B2A72D F1F117 E64444 D8C300
      "London, N >= 1": "#36B28D", # 32A683 A968E6
    
      "England, N == 0": "#CDC134",  # B2A72D F1F117 E64444 D8C300
      "England, N >= 1": "#36B28D" # 32A683 A968E6
    },
    interpolate="monotone",
    legend_field="domain",
    legend_view_style="legend-view",
    y_title="Frequency",
    alpha_suffix=", 2s",
    quartile_stroke_dash=[2, 1],
    include_legend=False,
    legend_concat_horizontally=False,
    legend_padding_outer=1,
    include_histograms=True,
    include_rules=True,
    significant_difference_color="#111111",
    insignificant_difference_color="#A62109", #"#DA2B0D", #"#DC462C",
    difference_label_align="left",
    difference_label_baseline="middle",
    difference_label_y=alt.Undefined,
    difference_label_style="result-text",
    variant_order=["London, N == 0", "England, N == 0", "London, N >= 1", "England, N >= 1"],
    vega_config_paths={"light": os.path.join(data_dir, "press-theme-light.json"), 
                       "dark": os.path.join(data_dir, "press-theme-dark.json")})

In [ ]:
contexts = run_domain_tasks([distribution_domains_eng], [plot_distributions], output_dir)